

**1. Chargement et étiquetage des données**






In [2]:
import pandas as pd

data = pd.read_excel('/content/balanced-reviews.xlsx')

# Ajout d'une colonne 'Sentiment' basée sur la note
def label_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Negative'

data['Sentiment'] = data['rating'].apply(label_sentiment)

# Affichage des données étiquetées
print(data[['rating', 'review', 'Sentiment']].head())


   rating                                             review Sentiment
0       2                  “ممتاز”. النظافة والطاقم متعاون.   Negative
1       5  استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...  Positive
2       5  استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...  Positive
3       1  “استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...  Negative
4       4  جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...  Positive


**2. Suppression du texte non-arabe**

In [4]:
import re

def remove_non_arabic(text):
    arabic_pattern = re.compile(r'[^\u0600-\u06FF\s]')
    return arabic_pattern.sub('', text)

data['Cleaned_Review'] = data['review'].apply(remove_non_arabic)

# Affichage des critiques nettoyées
print(data[['review', 'Cleaned_Review']].head())


                                              review  \
0                  “ممتاز”. النظافة والطاقم متعاون.    
1  استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...   
2  استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...   
3  “استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...   
4  جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...   

                                      Cleaned_Review  
0                      ممتاز النظافة والطاقم متعاون   
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ  
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...  
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه   
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...  


**3. Suppression de la ponctuation et des caractères spéciaux**

In [5]:
import string
import re

def remove_punctuation(text):
    arabic_punctuation = ['،', '؛', '؟', '٪', '…', 'ـ', '(', ')', '[', ']', '{', '}', '<', '>']
    all_punctuation = string.punctuation + ''.join(arabic_punctuation)

    # Supprimer la ponctuation (arabe + générale)
    text = text.translate(str.maketrans('', '', all_punctuation))
    return text

data['Cleaned_Review_No_Punctuation'] = data['Cleaned_Review'].apply(remove_punctuation)

# Afficher les résultats
print(data[['Cleaned_Review', 'Cleaned_Review_No_Punctuation']].head())


                                      Cleaned_Review  \
0                      ممتاز النظافة والطاقم متعاون    
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

                       Cleaned_Review_No_Punctuation  
0                      ممتاز النظافة والطاقم متعاون   
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ  
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...  
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه   
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...  


**4. La suppression des lettres répétées excessives**

In [6]:
import re

def remove_repeated_letters(text):
    # Remplacer 3 répétitions ou plus par 2
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    return text

data['Normalized_No_Repeated_Letters'] = data['Cleaned_Review_No_Punctuation'].apply(remove_repeated_letters)

# Afficher les résultats
print(data[['Cleaned_Review_No_Punctuation', 'Normalized_No_Repeated_Letters']].head())


                       Cleaned_Review_No_Punctuation  \
0                      ممتاز النظافة والطاقم متعاون    
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

                      Normalized_No_Repeated_Letters  
0                      ممتاز النظافة والطاقم متعاون   
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ  
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...  
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه   
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...  


**5. Standardisation des mots dialectaux en arabe littéraire**

In [7]:
# Dictionnaire de transformation du dialecte koweïtien vers l'arabe standard
dialect_to_standard_dict = {
    "شلون": "كيف",
    "زين": "جيد",
    "شفيك": "ما بك",
    "ليش": "لماذا",
    "وين": "أين",
    "شنو": "ماذا",
    "خوش": "ممتاز",
    "وايد": "كثير",
    "الحين": "الآن",
    "يبي": "يريد",
    "عشان": "لأن",
    "شوف": "انظر",
    "ماكو": "لا يوجد",
    "قاعد": "يفعل حالياً",
    "ماعليه": "لا بأس",
    "إنتي": "أنتِ",
    "إنت": "أنت",
    "أبوي": "أبي",
    "يمّه": "أمي",
    "مالت عليك": "عار عليك",
    "كلش": "أبداً",
    "تيي": "تأتي",
    "هني": "هنا",
    "بره": "خارج",
    "اشفيها": "ما بها",
    "أخيي": "يا أخي",
    "عطيني": "أعطني",
    "يمشي": "يسير",
    "يبيله": "يحتاج إلى",
    "حيل": "جداً",
    "طاحت": "سقطت",
    "إلحق": "اسرع",
    "توني": "للتو",
    "واحد": "شخص",
    "صار": "أصبح",
    "درب": "طريق",
    "ما يصير": "لا يمكن",
    "شال": "رفع",
    "عايش": "يعيش",
    "يلعن": "يسب",
    "يجوف": "يرى"
}

def dialect_to_standard(text):
    for dialect_word, standard_word in dialect_to_standard_dict.items():
        text = text.replace(dialect_word, standard_word)
    return text

data['Standardized_Review'] = data['Normalized_No_Repeated_Letters'].apply(dialect_to_standard)

# Afficher les résultats
print(data[['Normalized_No_Repeated_Letters', 'Standardized_Review']].head())


                      Normalized_No_Repeated_Letters  \
0                      ممتاز النظافة والطاقم متعاون    
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

                                 Standardized_Review  
0                      ممتاز النظافة والطاقم متعاون   
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ  
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...  
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه   
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...  


**6. Normalisation du texte arabe**

In [9]:
def normalize_arabic(text):
    text = text.replace('آ', 'ا')  # Remplacer 'آ' par 'ا'
    text = text.replace('ٱ', 'ا')  # Remplacer 'ٱ' par 'ا'
    text = text.replace('َ', '')   # Supprimer 'fatha'
    text = text.replace('ً', '')   # 'tanwin fatha'
    text = text.replace('ُ', '')   # 'damma'
    text = text.replace('ٌ', '')   # 'tanwin damma'
    text = text.replace('ِ', '')   # 'kasra'
    text = text.replace('ٍ', '')   # 'tanwin kasra'
    text = text.replace('ْ', '')   # 'sukun'
    text = text.replace('ّ', '')   # 'shadda'
    return text

data['Normalized_Review'] = data['Standardized_Review'].apply(normalize_arabic)

# Afficher les résultats
print(data[['Standardized_Review', 'Normalized_Review']].head())


                                 Standardized_Review  \
0                      ممتاز النظافة والطاقم متعاون    
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

                                   Normalized_Review  
0                      ممتاز النظافة والطاقم متعاون   
1   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ  
2  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...  
3   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه   
4  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...  


**7. Sauvegarde les données prétraitées dans un fichier Excel**

In [11]:
import pandas as pd
from google.colab import files

output_data = data[['no', 'Hotel name', 'rating', 'user type', 'room type', 'nights', 'Normalized_Review', 'Sentiment']]

print(output_data.head())

output_filename = '/content/hotel_reviews_preprocessed.xlsx'
output_data.to_excel(output_filename, index=False)

# Télécharger le fichier
files.download(output_filename)

print("Le fichier Excel a été sauvegardé sous le nom 'hotel_reviews_preprocessed.xlsx'.")


   no Hotel name  rating    user type                   room type  \
0   2    فندق 72       2  مسافر منفرد  غرفة ديلوكس مزدوجة أو توأم   
1   3    فندق 72       5          زوج  غرفة ديلوكس مزدوجة أو توأم   
2  16    فندق 72       5          زوج                           -   
3  20    فندق 72       1          زوج          غرفة قياسية مزدوجة   
4  23    فندق 72       4          زوج  غرفة ديلوكس مزدوجة أو توأم   

            nights                                  Normalized_Review  \
0  أقمت ليلة واحدة                      ممتاز النظافة والطاقم متعاون    
1  أقمت ليلة واحدة   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2      أقمت ليلتين  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3  أقمت ليلة واحدة   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4      أقمت ليلتين  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

  Sentiment  
0  Negative  
1  Positive  
2  Positive  
3  Negative  
4  Positive  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Le fichier Excel a été sauvegardé sous le nom 'hotel_reviews_preprocessed.xlsx'.
